# Executar um script de treinamento com SDK do Python

In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    #  Check if given  credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaaultAzureCredential not work.
    credential = InteractiveBrowserCredential()

In [ ]:
# Get a handle to the workspace
ml_client = MLClient(credential=credential )

In [ ]:
%%writefile src/sorvetes-training.py
import pandas as pd
import numpy as np
import sklearn.model_selection import train_test_split
import sklearn.linear_model import LogisticRegression
import sklearn.metrics import roc_auc_score


#load the sorvetes dataset
print("Loading Data ...")
sorvetes = pd.read_csv('vendas_sorvete.csv')

# separate feature and labels 
X, y = sorvetes[['Temperatura (°C)']].values, sorvetes['Vendas de Sorvete (Unidades)'].values

# split the data into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

# set regularization Hyperparameter
reg = 0.01

# train a logistic regression model 
print('Training a logistic regression model with regularization rate of',reg)
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy 

y_hat = model.predict(X_test)
acc = np.avarage(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test, y_scores[:, 1])
print('AUC: '+str(auc))

In [ ]:
# Importando as bibliotécas a serem utilizadas
import pandas as pd
import numpy as np
from datetime import datetime

#Criando uma função para validar se a entrada de datas está conforme o formato mencionado 
def valida_data():
    while True:
        try:
            data_range  = input("Digite o período de vendas a ser sorteado no formato de AAA/MM/DD")            
            if data_range:
                data = datetime.strptime(data_range,"%Y-%m-%d")
                return data
        except ValueError:
            print("Digite um formato válido para o perído escolhido")

data = valida_data()

In [ ]:
from azure.ai.ml import command 

# Configure Job
job = command(
    code ="./src", 
    command ="python sorvetes-training.py", 
    environment ="Azureml-sklearn-0.24-ubuntu18.04-py37-cpu@latest", 
    compute="cpu-cluster", 
    display_name="sorvetes-pythonv2-train",
    experiment_name="sorvetes-training"
)

# submit job 
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at",aml_url)